# Twitter API v2 to MongoDB

Functions for storing tweets and Twitter user data in MongoDB. Includes a function to gather historical data from a list of categorized accounts provided by the user in an Excel spreadsheet.

To do:
* Implement a function to gather only new tweets from a user or search;
* Fail elegantly in case of a typo or deleted account in the user-provided spreadsheet;
* Fail elegantly and enable a smooth restart in case of API errors;
* Implement logging.

In [6]:
import time
import json
import pandas as pd
import pprint
import pymongo
import searchtweets
from twitter_api import recent_search, user_lookup, user_tweets

In [60]:
%env BEARER_TOKEN= [Your Token]

env: BEARER_TOKEN=[Your Token]


In [8]:
def index_database():
    """Create indices for the MongoDB collections"""
    requested_accounts.create_index('user_id', unique = True)
    twitter_user_data.create_index('id', unique = True)
    collected_tweets.create_index('id', unique = True)
    collected_tweets.create_index([('text', 'text')], name = "tweet_text")

In [9]:
def parse_excel_2(input_file):
    """Reads Excel file and returns username and category as list of tuples"""
    input_data = pd.read_excel(input_file)
    excel_data = input_data.to_records(index=False)
    print(excel_data)
    return(excel_data)

In [10]:
# Retrieve list of user_ids, categories that have loaded = 0
def list_new_accounts():
    """Checks number of accounts in the database that do not have associated Twitter user IDs. 
    Returns a dictionary in the format {category1:[account1, account2], category2:[account3, account4]}"""
    
    # Report number of accounts with loaded == 0
    num_to_load = db.requested_accounts.count_documents({"loaded":0})
    print(f"Number of usernames to lookup: {num_to_load}")

    # Retrieve list of categories currently in the username database
    categories_in_use = db.requested_accounts.distinct('category')
    print(f"Categories in database: {categories_in_use}")

    # Create lists of unretrieved account by category
    account_lists = {}
    for item in categories_in_use:
        accounts =[]
        for entry in db.requested_accounts.find({"category":item, "loaded":0}):
            accounts.append(entry['user_id'])
        account_lists[item] = accounts

    print(account_lists)
    return(account_lists)

In [11]:
# Insert userdata returned by Twitter into MongoDB
def load_userdata(userdata, cat):
    new_userdata = json.loads(userdata)
    for entry in new_userdata['data']:
        try:
            entry['date_added'] = time.strftime("%Y/%m/%d")
            entry['time_added'] = time.strftime("%H:%M%:%S_%Y/%m/%d")
            entry['category'] = cat
            entry['hist_data_collected'] = 0
            twitter_user_data.insert_one(entry)
            
            # Write existing database to indicate userdata has been read
            username = entry['username']
            db.requested_accounts.update_one({"user_id":username},{'$set':{"loaded":1}})
            
        except:
            pass

In [12]:
# Get a user's historical tweets
def get_historical(user_id, username, cat, **kwargs):
    """Get tweets since Dec 4 for user_id and enter into MongodB"""
    print(f"Gathering up to 100 tweets for user {username}")
    if 'next_token' in kwargs:
        some_tweets = user_tweets.main(user_id, next_token = kwargs.get('next_token'))
    else:
        some_tweets = user_tweets.main(user_id)
        
    jsonified_tweets = json.loads(some_tweets)
    
    # If the user has tweeted in the time period specified, collect the tweets
    if 'data' in jsonified_tweets:
        for tweet in jsonified_tweets['data']:
            try:
                tweet['username'] = username
                tweet['category'] = cat
                collected_tweets.insert_one(tweet)
            except:
                pass
    else:
        pass
    
    # If there are more tweets to collect from the user, get the next batch
    if 'next_token' in jsonified_tweets['meta']:
        next_token = jsonified_tweets['meta']['next_token']
        time.sleep(2)
        get_historical(user_id, username, cat, next_token = next_token)
    else:
        # If finished, record that historical data has been collected for this user
        db.twitter_user_data.update_one({"id":user_id},{'$set':{"hist_data_collected":1}})
        return 0

In [8]:
# Load list of user_ids and categories into MongoDB
def load_excel_to_mongo():
    """Takes account_list.xlsx, adds new accounts into requested_accounts collection"""
    excel_data = parse_excel_2('account_list.xlsx')

    requested_accounts = db['requested_accounts']

    for item in excel_data:
        try:
            entry = {}
            user, category = item
            entry['user_id'] = user
            entry['category'] = category
            entry['loaded'] = 0 # has userdata been collected yet?
            requested_accounts.insert_one(entry)
        except:
            pass

In [ ]:
# Get new tweets
def get_new_tweets():
    """Gather new tweets for user since last tweet in DB"""
    

In [13]:
# Create database
client = pymongo.MongoClient()
db = client['GA_Runoffs']

# Create test userdata collection
requested_accounts = db['requested_accounts']
twitter_user_data = db['twitter_user_data']
collected_tweets = db['collected_tweets']

In [ ]:
load_excel_to_mongo()

In [ ]:
# Take accounts from list_new_accounts, query Twitter API for user info, enter into MongoDB
# TODO: Fail elegantly if typo in account list (bad request reutrns error 400)

account_lists = list_new_accounts()

for cat in list(account_lists.keys()):
    usernames = account_lists[cat]
    if len(usernames) >= 1:
        users = ','.join(usernames)
        users = 'usernames='+users
        print(f"User query string: {users}")

        userdata = user_lookup.main(users)

        load_userdata(userdata, cat)
    else:
        pass

In [8]:
# Return Twitter IDs, usernames, and categories of any users for whom historical data has not been collected
to_collect = db.twitter_user_data.find({'hist_data_collected':0})
to_collect_list = []

for item in to_collect:
    userid = item['id']
    username = item['username']
    category = item['category']
    data = [userid, username, category]
    to_collect_list.append(data)
    
print(to_collect_list)

# Get data of uncollected users
for item in to_collect_list:
    get_historical(item[0], item[1], item[2])
    
    
    

[['2863210809', 'sendavidperdue', 'candidates'], ['521747968', 'ossoff', 'candidates'], ['29495695', 'KLoeffler', 'candidates'], ['1200451909406121984', 'SenatorLoeffler', 'candidates'], ['1221242033530195970', 'ReverendWarnock', 'candidates'], ['74568060', 'GeorgiaDemocrat', 'dem_state'], ['1726021093', 'DPGChair', 'dem_state'], ['36424664', 'NikemaWilliams', 'dem_state'], ['19149039', 'DavidShafer', 'gop_state'], ['74482441', 'GaRepublicans', 'gop_state'], ['888452059493806081', 'CCDCGeorgia', 'dem_county'], ['19671185', 'cherokeedems', 'dem_county'], ['129710589', 'CobbDemocrats', 'dem_county'], ['4750220728', 'cowetadems', 'dem_county'], ['1434486529', 'DekalbGADems', 'dem_county'], ['2363031012', 'DCDP_GA', 'dem_county'], ['350599621', 'GAFayetteDems', 'dem_county'], ['724436479649320961', 'ForsythDemGA', 'dem_county'], ['240303484', 'GwinnettDems', 'dem_county'], ['823326298579881985', 'HallCountyDem', 'dem_county'], ['1379699762', 'HenryDems', 'dem_county'], ['2441166067', 'HoCo

In [36]:
index_database()

In [3]:
test = recent_search.main()

200
{
    "data": [
        {
            "author_id": "198034921",
            "conversation_id": "1344295684732841986",
            "created_at": "2020-12-30T14:54:01.000Z",
            "entities": {
                "annotations": [
                    {
                        "end": 82,
                        "normalized_text": "Mitch McConnell",
                        "probability": 0.9932,
                        "start": 68,
                        "type": "Person"
                    }
                ],
                "mentions": [
                    {
                        "end": 19,
                        "start": 3,
                        "username": "fairfightaction"
                    },
                    {
                        "end": 41,
                        "start": 31,
                        "username": "KLoeffler"
                    },
                    {
                        "end": 61,
                        "start": 46,
                     

In [14]:
# Return tweets by search query
def search_recent_tweets(query: str, category: str, counter: int, **kwargs):
    """Query recent tweets and add to MongoDB with a category label"""
    if counter >=10000:
        print("Gathered maximum number of tweets")
        return 0
    
    else: 
        print(f"Gathering up to 10,000 tweets for query {query}. Approx {counter} collected so far")

        if 'next_token' in kwargs:
            some_tweets = recent_search.main(query, next_token = kwargs.get('next_token'))
            counter += 100
        else:
            some_tweets = recent_search.main(query)
            counter += 100

        jsonified_tweets = json.loads(some_tweets)

        # If the search returned results, collect the tweets
        if 'data' in jsonified_tweets:
            for tweet in jsonified_tweets['data']:
                try:
                    tweet['query'] = query
                    tweet['category'] = category
                    collected_tweets.insert_one(tweet)
                except:
                    pass
        else:
            pass

        # If there are more tweets to collect from the user, get the next batch
        if 'next_token' in jsonified_tweets['meta']:
            next_token = jsonified_tweets['meta']['next_token']
            time.sleep(2)
            search_recent_tweets(query, category, counter, next_token = next_token)
        else:
            print(f"Completed query {query} with approximately {counter} tweets collected.")
            return 0
        
    return 0

In [17]:
# Search Tweets that mention @ReverendWarnock and that are not retweets
search_recent_tweets("(%40ReverendWarnock) -is:retweet", "hashtag_search", 0)

Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 0 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 200 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 300 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 400 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 500 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 600 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 700 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 800 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 900 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. 

Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8000 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8100 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8200 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8300 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8400 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8500 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8600 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8700 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 8800 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -i

Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 15900 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16000 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16100 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16200 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16300 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16400 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16500 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16600 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 16700 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWa

Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 23800 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 23900 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24000 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24100 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24200 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24300 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24400 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24500 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWarnock) -is:retweet. Approx 24600 collected so far
Gathering up to 100,000 tweets for query (%40ReverendWa

KeyboardInterrupt: 

In [13]:
# Search Tweets that mention @ReverendWarnock and that are not retweets
search_recent_tweets("%23holdtheline -is:retweet", "holdtheline_ecosystem", 0)

Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 0 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 200 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 300 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 400 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 500 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 600 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 700 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 800 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 900 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 1000 collected so far
Gathering up to 100,000 tweets 

Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 8500 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 8600 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 8700 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 8800 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 8900 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 9000 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 9100 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 9200 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 9300 collected so far
Gathering up to 100,000 tweets for query %23holdtheline -is:retweet. Approx 9400 collected so far
Gathering up to 100,

KeyboardInterrupt: 

In [14]:
# Search Tweets that use hashtag #DemCastGA and that are not retweets
search_recent_tweets("%23demcastga -is:retweet", "demcast_ecosystem", 0)

Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 0 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 200 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 300 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 400 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 500 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 600 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 700 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 800 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 900 collected so far
Gathering up to 100,000 tweets for query %23demcastga -is:retweet. Approx 1000 collected so far
Gathering up to 100,000 tweets for query %23demcast

KeyboardInterrupt: 

In [25]:
# Search tweets that use hashtags #GAPol or #GASen and that are not rewtweets
search_recent_tweets("(%23gapol OR %23gasen) -is:retweet", "hashtag_search", 0)

Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 0 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 100 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 200 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 300 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 400 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 500 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 600 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 700 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 800 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is

Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 7900 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8000 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8100 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8200 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8300 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8400 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8500 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8600 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %23gasen) -is:retweet. Approx 8700 collected so far
Gathering up to 10,000 tweets for query (%23gapol OR %2

0

In [25]:
# Show categories currently in the collected_tweets collection
db.collected_tweets.distinct("category")

['candidates',
 'dem_county',
 'dem_state',
 'demcast_ecosystem',
 'gop_county',
 'gop_state',
 'hashtag_search',
 'holdtheline_ecosystem']

In [58]:
# Get ID of most recent tweet for a query
def get_most_recent_entry(query):
    most_recent = db.collected_tweets.find(query).sort("id",pymongo.DESCENDING).limit(1)
    most_recent_df = pd.DataFrame(list(most_recent))
    return(most_recent_df.iloc[0]['id'])

In [59]:
# Get Twitter ID of most recent Tweet collected for category hashtag_search
get_most_recent_entry({"category":{"$eq":"hashtag_search"}})

'1346573423443652609'

'1346573423443652609'

""


1346573423443652609
